### IMPORTS

In [1]:
import json
import base64
import hashlib
import pkg_resources.py2_warn

In [2]:
from web3 import Web3

In [3]:
print('Booting up...')

Booting up...


### LOAD DATA FROM EXTERNAL FILES

In [4]:
with open('settings.json') as json_file:
    settings = json.load(json_file)

In [5]:
with open('identifier.json') as json_file:
    device = json.load(json_file)

In [6]:
with open('latest.json') as json_file:
    latest = json.load(json_file)

### IPFS & BLOCKCHAIN GATEWAY CONNECTIONS

In [7]:
class gateway:
    def __init__(self, foo):
        self.host = foo['host']
        self.port = foo['port']

In [8]:
blockchain = gateway(settings['gateways']['blockchain'])

In [9]:
ipfs = gateway(settings['gateways']['ipfs'])

In [10]:
print('Gateways set!')

Gateways set!


### BLOCKCHAIN PUBLIC/PRIVATE KEYS

In [11]:
class keypair:
    def __init__(self, foo):
        self.public = foo['public']
        self.private = foo['private']

In [12]:
keys = keypair(settings['keys'])

In [13]:
print('Keys set!')

Keys set!


### HASH DEVICE PARAMS FOR IDENTIFIER

In [14]:
def hash_id(data):
    
    # CONVERT TO BYTES
    to_bytes = str.encode(str(data))
    
    # HASH ENCODED DATA
    hashed = hashlib.sha224(to_bytes).hexdigest()
    
    return hashed

In [64]:
device_id = hash_id(device)

In [61]:
print('Device identifier hashed!')

Device identifier hashed!


### CONNECT TO CHAIN VIA WEBSOCKET

In [16]:
web3 = Web3(Web3.WebsocketProvider('ws://' + blockchain.host + ':' + blockchain.port))

In [17]:
if web3.isConnected():
    print('Connected with gateway!')
else:
    print('Could not connect to gateway!')

Connected with gateway!


### CONSTRUCT MANAGER CONTRACT

In [18]:
class manager:
    
    # ON LOAD..
    def __init__(self, block):
        
        # CONSTRUCT USABLE CONTRACT
        self.contract = web3.eth.contract(
            address = block['address'],
            abi = block['abi']
        )
        
        # SET ADDRESS REFERENCE
        self.address = block['address']
    
    # READ FROM CONTRACT
    async def read(self, details):
        
        # WITH PARAMS
        if ('params' in details):
            return self.contract.functions[details['func']](details['params']).call()
        
        # WITHOUT PARAMS
        else:
            return self.contract.functions[details]().call()
    
    # WRITE TO CONTRACT
    async def write(self, details):
        try:
            
            # CREATE BASE TRANSACTION
            tx = {
                'from': keys.public,
                'to': self.contract.address,
                'data': self.contract.encodeABI(
                    fn_name = details['func'],
                    args = details['params']
                )
            }
            
            # ESTIMATE GAS VALUE & STITCH IN REMAINING PROPS
            tx['gas'] = web3.eth.estimateGas(tx)
            tx['gasPrice'] = web3.toWei(1, 'gwei')
            tx['nonce'] = web3.eth.getTransactionCount(keys.public)

            # SIGN TRANSCTION WITH PRIVATE KEY
            signed = web3.eth.account.sign_transaction(tx,
                private_key = keys.private
            )

            # SEND THE TRANSACTION
            web3.eth.sendRawTransaction(signed.rawTransaction)

            # SUCCESS
            return True
        
        # IF THE TRANSACTION IS REVERTED, SHOW ERROR
        except ValueError as error:
            return error

### CONTRACT REFERENCES

In [19]:
user_manager = manager(latest['usermanager'])

In [20]:
device_manager = manager(latest['devicemanager'])

In [21]:
task_manager = manager(latest['taskmanager'])

In [22]:
token_manager = manager(latest['tokenmanager'])

In [23]:
print('Manager contracts set!')

Manager contracts set!


### INIT MANAGER CONTRACTS

In [24]:
async def init_managers():
    
    # FLEXIBLE PARAMS
    token_price = 5000
    task_fee = 5
    
    # TOKEN MANAGER
    await token_manager.write({
        'func': 'init',
        'params': [
            token_price,
            task_manager.address
        ]
    })
    
    # TASK MANAGER
    await task_manager.write({
        'func': 'init',
        'params': [
            task_fee,
            user_manager.address,
            device_manager.address,
            token_manager.address
        ]
    })
    
    # USER MANAGER
    await user_manager.write({
        'func': 'init',
        'params': [task_manager.address]
    })
    
    # DEVICE MANAGER
    await device_manager.write({
        'func': 'init',
        'params': [
            user_manager.address,
            task_manager.address
        ]
    })
    
    return True

### PERFORM INITIALIZATION -- IF NECESSARY

In [34]:
# CHECK STATUS
initialized = await token_manager.read('initialized')

In [35]:
# INIT HAS BEEN PERFORMED
if (initialized):
    print('Manager contracts have already been initialized!')
    
# INIT NOW
else:
    result = await init_managers()
    
    if (result):
        print('Initializing manager contracts!')
    else:
        print('Could not initialize manager contracts!')

Manager contracts have already been initialized!


### REGISTER A USER -- IF NECESSARY

In [56]:
async def register_user():
    
    # REGISTER ETH WALLET
    await user_manager.write({
        'func': 'add',
        'params': []
    })
    
    return True

In [59]:
user_check = await user_manager.read({
    'func': 'fetch',
    'params': keys.public
})

In [60]:
# USER NOT FOUND
if (user_check == '0x0000000000000000000000000000000000000000'):
    
    # REGISTER NOW
    result = await register_user()
    
    if (result):
        print('Registered your wallet!')
    
# USER FOUND
else:
    print('Your wallet is already registered!')

Your wallet is already registered!


### REGISTER A DEVICE -- IF NECESSARY

In [73]:
async def register_device(name):
    
    # REGISTER ETH WALLET
    await device_manager.write({
        'func': 'add',
        'params': [device_id, name]
    })
    
    return True

In [74]:
device_check = await device_manager.read({
    'func': 'fetch_device',
    'params': device_id
})

In [75]:
# USER NOT FOUND
if (device_check == '0x0000000000000000000000000000000000000000'):
    
    # REGISTER NOW
    result = await register_device('foo')
    
    if (result):
        print('Registered this device!')
    
# USER FOUND
else:
    print('This device is already registered!')

This device is already registered!
